# Basic Usage of runnable_family

This notebook describes the basic usage of the `runnable_family` package.
This package provides utilities implemented according to the `Runnable` protocol of `langchain`.
This notebook will give you an idea of what `langchain` components you can build with the `runnable_family` package.

## Preparation

First of all, you need to setup a virtual environment.

`setup.sh` helps you setup.

Altenatively,

```bash
python -m venv venv
venv/bin/python -m pip install -e ..[openai]
venv/bin/python -m pip install jupyter jupyterlab ipykernel
echo "OPENAI_API_KEY={HERE_IS_YOUR_API_KEY}" > .env
```

## Libs

In [1]:
from datetime import datetime as dt
from collections import Counter
from operator import attrgetter

from dotenv import load_dotenv
from langchain_core.runnables import RunnableLambda, RunnablePassthrough
from langchain_openai import ChatOpenAI
from langchain.output_parsers import PydanticOutputParser
from langchain.pydantic_v1 import BaseModel
from langchain.prompts import ChatPromptTemplate

from runnable_family.basic import RunnableConstant, RunnableAdd, RunnableLog
from runnable_family.loopback import RunnableLoopback
from runnable_family.gacha import RunnableGacha
from runnable_family.runnable_diff import RunnableDiff
from runnable_family.self_consistent import RunnableSelfConsistent
from runnable_family.self_refine import RunnableSelfRefine
from runnable_family.self_translate import RunnableSelfTranslate

In [2]:
load_dotenv()

True

## Example

### `runnable_family.basic`

This subpackage provides basic utilities for the `Runnable` protocol which are expected to be used in anothre class in `runnable_family`.

The most fundamental class in this subpackage is `RunnableConstant`.

This class is a `Runnable` class which outputs a constant value for any inputs.

In [3]:
runnable_constant = RunnableConstant(constant=1)
print('runnable_constant.invoke(None) = ', runnable_constant.invoke(None))
print('runnable_constant.invoke(2) = ', runnable_constant.invoke(2))

runnable_constant.invoke(None) =  1
runnable_constant.invoke(2) =  1


Here is another fundamental class, `RunnableAdd`.

This class is a `Runnable` class which add a constant value to the input value.

In [4]:
runnable_add = RunnableAdd(constant=1)
print('runnable_add.invoke(10) = ', runnable_add.invoke(10))
print('runnable_add.invoke(100) = ', runnable_add.invoke(100))

runnable_add.invoke(10) =  11
runnable_add.invoke(100) =  101


Note that you can specify the order of addition with `left` argument.

If `left` is True, which is the default value, an instance of `RunnableAdd` adds the constant value to the input value from the right side.

In other words, the `left` argument can be interpreted as a parameter that controls whether the input value is counted from the left to a constant value.

In [5]:
runnable_add_left = RunnableAdd(constant='a', left=True)
runnable_add_right = RunnableAdd(constant='a', left=False)

print(f'runnable_add_left.invoke("A") = ', runnable_add_left.invoke('A'))
print(f'runnable_add_right.invoke("A") = ', runnable_add_right.invoke('A'))

runnable_add_left.invoke("A") =  Aa
runnable_add_right.invoke("A") =  aA


We sometimes want to use `print` or `logger` between `Runnable` classes for debugging.

Of course, you can use `LANGCHAIN_TRAICING_V2` environment variable, which is described in https://python.langchain.com/v0.1/docs/expression_language/why/#logging .

However, `RunnableLog` is an intiutive component for debugging.

In [6]:
log_runnable = RunnableLog(print)
chain = (
    RunnableAdd(1)
    | RunnableAdd(2)
    | log_runnable
    | RunnableAdd(3)
    | log_runnable
)
chain.invoke(0)

3
6


6

### `runnable_family.loopback`

This subpackage provides `RunnableLoopback`, which enables to loopback the output of a `Runnable` class to the input of the same class.

Because such a runnable is not provided in `langchain`, `RunnableLoopback` is one of the central classes in `runnable_family`.

Here is an example of `RunnableLoopback`.

This example provides a `Runnable` class which counts up the input value by 1.

In [7]:
runnable_loopback = RunnableLoopback(
    runnable=(
        RunnableAdd(constant=1)  # calulate x + 1 where x is the input
        | RunnableLambda(lambda x: print(f'current output of the runnable: {x}') or x)  # print x and return x
    ),
    condition=RunnableLambda(lambda y: y < 10),  # only loopback if y < 10 where y is the output of the runnable
    loopback=RunnablePassthrough(),  # runnable which transform the output of the runnable to the input of the runnable
)

In [8]:
runnable_loopback.invoke(0)

current output of the runnable: 1
current output of the runnable: 2
current output of the runnable: 3
current output of the runnable: 4
current output of the runnable: 5
current output of the runnable: 6
current output of the runnable: 7
current output of the runnable: 8
current output of the runnable: 9
current output of the runnable: 10


10

`get_graph` method helps you understand the structure of `RunnableLoopback`.

In [9]:
print(runnable_loopback.get_graph().draw_ascii())

                                   +---------------+                          
                                   | LoopbackInput |                          
                                   +---------------+                          
                                            *                                 
                                            *                                 
                                            *                                 
                                    +------------+                            
                                    | _add_input |**                          
                                    +------------+  ******                    
                                 ***                      *****               
                              ***                              *****          
                            **                                      ******    
                  +--------------+                  

`RunnableLoopback` also provides a class method which create an instance of `RunnableLoopback` with a specified number of loopbacks.

The following example let you know how to create a runnable which prints the current time 5 times.

In [10]:
runnable_loopback_with_n_times = RunnableLoopback.with_n_loop(
    n=5,
    runnable=(
        RunnableLambda(lambda *args: dt.now())  # get the current time
        | RunnableLambda(lambda x: print(f'current output of the runnable: {x}') or x)  # print x and return x
    ),
    loopback=RunnablePassthrough(),  # runnable which transform the output of the runnable to the input of the runnable
)

In [11]:
runnable_loopback_with_n_times.invoke(1)

current output of the runnable: 2024-05-20 23:52:13.702129
current output of the runnable: 2024-05-20 23:52:13.912347
current output of the runnable: 2024-05-20 23:52:14.134929
current output of the runnable: 2024-05-20 23:52:14.346604
current output of the runnable: 2024-05-20 23:52:14.574298


datetime.datetime(2024, 5, 20, 23, 52, 14, 574298)

### `runnable_family.gacha`

This subpackage provides `RunnableGacha`.

`RunnableGacha` helps you run a stochastic runnable like `ChatOpenAI` without `seed` many times.

The following example shows how to run `ChatOpenAI(model='gpt-4-turbo', temperature=1).invoke` 3 times.

In [12]:
runnable_gacha = RunnableGacha(
    runnable=(
        ChatOpenAI(model='gpt-4-turbo', temperature=1)
        | RunnableLambda(attrgetter('content'))
    ),
    n=3,
)

In [13]:
runnable_gacha.invoke('What is the meaning of life?')

["The question of the meaning of life has been a central and enduring inquiry in philosophy, theology, science, and literature. It asks what the purpose of life is, or why we exist. The answer to this question varies greatly depending on cultural, individual, philosophical, and religious contexts. Here are some perspectives:\n\n1. **Religious and Spiritual Perspectives**: Many theistic religions, such as Christianity, Islam, and Hinduism, often claim that life's purpose is to serve and honor God. In these views, life is a test, a preparation for the afterlife, or a chance to grow spiritually.\n\n2. **Philosophical Perspectives**: Philosophers have approached this question in varied ways:\n   - **Existentialists** like Jean-Paul Sartre and Albert Camus argue that life initially has no meaning, and it is up to each individual to give their own life meaning through their choices and actions.\n   - **Nihilists**, such as Friedrich Nietzsche at times, suggest that life inherently has no mea

### `runnable_family.runnable_diff`

This subpackage provides `RunnableDiff`.

`RunnableDiff` helps you compare the outputs of two runnables.

Here is an example that points out the shortcomings of the output of 'gpt-3.5-turbo' relative to the output of 'gpt-4-turbo'.

In [14]:
runnable_diff = RunnableDiff(
    runnable1=(
        ChatOpenAI(model='gpt-4-turbo', temperature=0.8, model_kwargs={'seed': 1000})
        | RunnableLambda(attrgetter('content'))
    ),
    runnable2=(
        ChatOpenAI(model='gpt-3.5-turbo', temperature=0.8, model_kwargs={'seed': 1000})
        | RunnableLambda(attrgetter('content'))
    ),
    diff=(
        RunnableLambda(lambda itr: '\n'.join([
            f'1. ```\n{itr[0]}\n```',
            f'2. ```\n{itr[1]}\n```',
            '\n',
            'The former is thought to be better than the latter.',
            'What do you think is insufficient in the latter?',
        ]))
        | RunnableLambda(lambda x: print(x) or x)
        | ChatOpenAI(model='gpt-4-turbo', temperature=0.8, model_kwargs={'seed': 1234})
    )
)

In [15]:
print(runnable_diff.invoke('What is the meaning of life?').content)

1. ```
The question of the meaning of life has been a central topic in philosophy, religion, and science for centuries, and it encompasses a broad spectrum of perspectives.

1. **Philosophical Perspectives**: Philosophers have debated the meaning of life from various angles. Some existentialists, like Jean-Paul Sartre, argue that life has no inherent meaning, and it is up to each individual to give meaning to their own life through their choices and actions. In contrast, philosophers like Aristotle have posited that the purpose of life is to achieve eudaimonia, often translated as "flourishing" or "happiness," through the cultivation of virtues.

2. **Religious Perspectives**: Most religions offer their own answers to the meaning of life:
   - **Christianity** traditionally teaches that the purpose of life is to love and serve God and to receive eternal life through Jesus Christ.
   - **Islam** similarly emphasizes submission to Allah's will and living according to the Quran's teaching

### `runnable_family.self_consistent`

This subpackage provides `RunnableSelfConsistent`.

`RunnableSelfConsistent` is a `Runnable` class which implements "self-consisntent" approach.

"self-consisntent" approach, suggested in https://arxiv.org/pdf/2203.11171.pdf, selects the most consistent answer by marginalizing out a diverse set of reasoning paths.

The following example is equievalent to the example in Figure 1 in https://arxiv.org/pdf/2203.11171.pdf .

In [16]:
class IntegerAnswer(BaseModel):
    answer: int


output_parser = PydanticOutputParser(pydantic_object=IntegerAnswer)
prompt = ChatPromptTemplate.from_messages([
    ("system", '{format_instruction}'),
    ("human", "{input}"),
])

runnable_self_consistent = (
    RunnablePassthrough().assign(format_instruction=RunnableConstant(output_parser.get_format_instructions()))
    | prompt
    | RunnableSelfConsistent(
        runnables=[
            ChatOpenAI(model='gpt-4-turbo', temperature=0.8, model_kwargs={'seed': 1000}) | output_parser | RunnableLambda(attrgetter('answer')),
            ChatOpenAI(model='gpt-4-turbo', temperature=0.8, model_kwargs={'seed': 1235}) | output_parser | RunnableLambda(attrgetter('answer')),
            ChatOpenAI(model='gpt-3.5-turbo', temperature=0.8, model_kwargs={'seed': 2500}) | output_parser | RunnableLambda(attrgetter('answer')),
        ],
        aggregate=(
            RunnableLambda(lambda itr: print(itr) or itr)  # print the outputs of the runnables
            | RunnableLambda(lambda itr: Counter(itr).most_common(1)[0][0])  # count the most common answer
        ),
    )
)

In [17]:
runnable_self_consistent.invoke(dict(input='\n'.join([
    'Q: If there are 3 cars in the parking lot and 2 more cars arrive, how many cars are in the parking lot?',
    'A: There are 3 cars in the parking lot already. 2 more arrive. Now there are 3 + 2 = 5 cars. The answer is 5.',
    '\n',
    'Q: Janet\'s ducks lay 16 eggs per day. '
    'She eats three for breakfast every morning and bakes muffins for her friends every day with four. '
    'She sells the remainder for $2 per egg. How much does she make every day?',
    'A: ',
])))

[18, 18, 18]


18

### `runnable_family.self_refine`

This subpackage provides `RunnableSelfRefine`.

`RunnableSelfRefine` is a `Runnable` which implements 'self-refine' approach, suggested in https://arxiv.org/pdf/2303.17651.pdf .

`RunnableSelfRefine` helps you refine the output automatically.

The following example is similar to the example in Figure 1 (d)(e)(f) in https://arxiv.org/pdf/2303.17651.pdf .

In this example, after generating a python code to calculate the sum of 1 to $n$, it is automatically refined in a point of view of the efficiency.

In [18]:
runnable_self_refine = RunnableSelfRefine(
    runnable=(
        ChatOpenAI(model='gpt-3.5-turbo', temperature=0.8, model_kwargs={'seed': 1000})
        | RunnableLambda(lambda x: print('==========\n'+x.content) or x)  # print the output
    ),
    feedback=(
        ChatPromptTemplate.from_messages([(
            'human',
            '\n'.join([
                'The user requires a code that satisfies the following',
                '```',
                '{input}',
                '```',
                'The service is attempting to return the following code in response to the above request:',
                '```',
                '{output}',
                '```',
                '\n',
                'Please point out the issues with this code in terms of computational efficiency.',
            ])
        )])
        | ChatOpenAI(model='gpt-3.5-turbo', temperature=0.8, model_kwargs={'seed': 1234})
        | RunnableLambda(lambda x: print('==========\n'+x.content) or x)  # print the feedback
    ),
    refine=(
        ChatPromptTemplate.from_messages([(
            'human',
            '\n'.join([
                'The user requires a code that satisfies the following',
                '```',
                '{input}',
                '```',
                'The service is attempting to return the following code in response to the above request:',
                '```',
                '{output}',
                '```',
                'For the above output code, the user give the following feedback:',
                '```',
                '{feedback}',
                '```',
                '\n',
                'Please refine the code to address the user\'s feedback.',
            ])
        )])
        | ChatOpenAI(model='gpt-3.5-turbo', temperature=0.8, model_kwargs={'seed': 1234})
        | RunnableLambda(lambda x: print('==========\n'+x.content) or x)  # print the refined code
    ),
    input_key='input',  # key to access the input. In this case, the default value will be used.
    output_key='output',  # key to access the output. In this case, the default value will be used.
    feedback_key='feedback',  # key to access the feedback. In this case, the default value will be used.
) | RunnableLambda(attrgetter('content'))

In [19]:
runnable_self_refine.invoke('Generate a python code which calculates the sum of 1, 2, ..., n.')

def sum_of_numbers(n):
    sum = 0
    for i in range(1, n+1):
        sum += i
    return sum

n = 10
result = sum_of_numbers(n)
print(f"The sum of numbers from 1 to {n} is: {result}")
One potential issue with this code in terms of computational efficiency is that it uses a simple loop to iterate through the numbers from 1 to n and calculate the sum. This approach has a time complexity of O(n) since it involves iterating through n numbers.

A more efficient approach would be to use the formula for the sum of the first n natural numbers, which is (n*(n+1))/2. This formula has a time complexity of O(1) as it directly calculates the sum without the need for iteration.

Therefore, for large values of n, using the formula for the sum of natural numbers would be more computationally efficient than the provided code with a loop.
Here is a refined version of the code that calculates the sum of 1, 2, ..., n using the formula (n*(n+1))/2 for better computational efficiency:

```python
def sum_o

'Here is a refined version of the code that calculates the sum of 1, 2, ..., n using the formula (n*(n+1))/2 for better computational efficiency:\n\n```python\ndef sum_of_numbers(n):\n    return (n*(n+1)) // 2\n\nn = 10\nresult = sum_of_numbers(n)\nprint(f"The sum of numbers from 1 to {n} is: {result}")\n```\n\nThis code directly calculates the sum of numbers from 1 to n using the formula for the sum of the first n natural numbers, which has a time complexity of O(1) and is more computationally efficient for large values of n compared to iterating through the numbers.'

### `runnable_family.self_translate`

This subpackage provides `RunnableSelfTranslate` based on https://arxiv.org/pdf/2308.01223.pdf .

`RunnableSelfTranslate` enables you to the following steps:

1. translate the input into another language;
2. infer the output with the translated input;
3. translate the output into the original language;

Here is an example of `RunnableSelfTranslate` with a Japanese traditional math problem.

In [20]:
runnable_self_translate = RunnableSelfTranslate(
    translater=(
        RunnableLambda(lambda x: f'Translate the following text to Japanese:\n```\n{x}\n```')
        | ChatOpenAI(model='gpt-3.5-turbo', temperature=0.8, model_kwargs={'seed': 1000})
        | RunnableLambda(attrgetter('content'))
        | RunnableLambda(lambda x: print('==========\n'+x) or x)
    ),
    inverse_translater=(
        RunnableLambda(lambda x: f'Translate the following text to English:\n```\n{x}\n```')
        | ChatOpenAI(model='gpt-3.5-turbo', temperature=0.8, model_kwargs={'seed': 1000})
        | RunnableLambda(attrgetter('content'))
        | RunnableLambda(lambda x: print('==========\n'+x) or x)
    ),
    runnable=(
        ChatOpenAI(model='gpt-3.5-turbo', temperature=0.8, model_kwargs={'seed': 1000})
        | RunnableLambda(attrgetter('content'))
        | RunnableLambda(lambda x: print('==========\n'+x) or x)
    ),
)

In [21]:
runnable_self_translate.invoke(
    'There are a total of 100 vines and turtles. '
    'If the total number of legs is 274, how many cranes and turtles are there? '
    'Use "Tsuru-Kame zan" step-by-step.'
)

合計100本のツルとカメがいます。合計の足の数が274本の場合、ツルとカメは合計で何匹いますか？「ツルカメ算」を使ってステップバイステップで解いてください。
「ツルカメ算」とは、ツルが2本、カメが4本の足を持つと仮定して、それぞれの動物の数を求める方法です。

1. ツルの数をx、カメの数をyとすると、以下の2つの方程式が成り立ちます。
x + y = 100 （合計の動物の数が100匹）
2x + 4y = 274 （合計の足の数が274本）

2. 上記の方程式を解くために、まず1番目の方程式を変形してxをyについて表します。
x = 100 - y

3. 次に、2番目の方程式にxの代入を行います。
2(100 - y) + 4y = 274
200 - 2y + 4y = 274
2y = 74
y = 37

4. カメの数が37匹なので、ツルの数は以下のようにして求めます。
x = 100 - y
x = 100 - 37
x = 63

したがって、ツルとカメの合計は63匹 + 37匹 = 100匹となります。
The "Crane and Tortoise Puzzle" is a method for determining the number of each animal, assuming that a crane has 2 legs and a tortoise has 4 legs.

1. Let x be the number of cranes and y be the number of tortoises, then the following two equations hold.
x + y = 100 (total number of animals is 100)
2x + 4y = 274 (total number of legs is 274)

2. To solve the above equations, first we rearrange the first equation to express x in terms of y.
x = 100 - y

3. Next, we substitute x into the second equation.
2(100 - y) + 4y = 274
2

'The "Crane and Tortoise Puzzle" is a method for determining the number of each animal, assuming that a crane has 2 legs and a tortoise has 4 legs.\n\n1. Let x be the number of cranes and y be the number of tortoises, then the following two equations hold.\nx + y = 100 (total number of animals is 100)\n2x + 4y = 274 (total number of legs is 274)\n\n2. To solve the above equations, first we rearrange the first equation to express x in terms of y.\nx = 100 - y\n\n3. Next, we substitute x into the second equation.\n2(100 - y) + 4y = 274\n200 - 2y + 4y = 274\n2y = 74\ny = 37\n\n4. Since the number of tortoises is 37, we can determine the number of cranes as follows.\nx = 100 - y\nx = 100 - 37\nx = 63\n\nTherefore, the total number of cranes and tortoises is 63 cranes + 37 tortoises = 100 animals.'